## Simulator (fleet of uavs)

This notebook serves to get familiar with the simulator

In [3]:
from config import Color
from helpers import clean
from helpers.coordinates import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)
from simulator.gazebo.config import GazWP
from simulator.QGroundControl.config import QGCWP

clean()

## Simulation Configuration

We can easily extend the configuration from Notebook 2 to multiple UAVs. 
We are also adding here a mission delay in secons.

In [4]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]
colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Visualizer

Extending the visualizer configuration to multiple UAVs is also straightforward.

## Gazebo

In [5]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)

gaz= Gazebo(gaz_config,gra_origin)

origin_gaz= GazWP(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)
gaz.markers.append(origin_gaz)

gaz.show()

## QGroundControl

In [6]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)

qgc=QGC(qgc_config)
origin_qgc = QGCWP(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)

qgc.markers.append(origin_qgc)
qgc.show()

## No Visualizer

In [7]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
    
novis=NoVisualizer(novis_config)

## Launch Simulation

**Here we configure the simulation:**
* **visualizers**: Select the desired visualizer for the simulation (only one visualizer at a time is currently supported).    
* **missions**: List of missions for each UAV. Although advanced, customized missions can be built using the `Mission` class defined in the QGroundControl configuration script, here we use the QGroundControl configuration builder for simplicity.  
* **terminals**: Processes to display in the terminal. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **suppress_output**: Processes whose output should be suppressed. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **verbose**: Verbosity level (0 = no information, 1 = normal information, 2 = include debugging information).  
* **other arguments**: More advanced options that allow further customization.  


In [8]:
qgc_config.vehicles

[QGCVehicle(home=GRAPose(lat=np.float64(-35.36319280060458), lon=np.float64(149.1652241), alt=np.float64(1.7697898478799932e-05), heading=0.0), mission=Mission(traj=[QGCWP(name='wp_0', pos=GRA(lat=np.float64(-35.36319280071091), lon=np.float64(149.1652241), alt=np.float64(5.000017697971329)), color=blue), QGCWP(name='wp_1', pos=GRA(lat=np.float64(-35.36310266784984), lon=np.float64(149.1652241), alt=np.float64(5.000049161251535)), color=blue), QGCWP(name='wp_2', pos=GRA(lat=np.float64(-35.36310266779976), lon=np.float64(149.16533413140553), alt=np.float64(5.000056990890237)), color=blue), QGCWP(name='wp_3', pos=GRA(lat=np.float64(-35.363192800660826), lon=np.float64(149.16533413152786), alt=np.float64(5.000025528639029)), color=blue), QGCWP(name='wp_4', pos=GRA(lat=np.float64(-35.36319280071091), lon=np.float64(149.1652241), alt=np.float64(5.000017697971329)), color=blue)], delay=0, land=True, speed=5.0)),
 QGCVehicle(home=GRAPose(lat=np.float64(-35.363327999887304), lon=np.float64(149

In [9]:
simulator = Simulator(
    gra_origin=gra_origin,
	visualizers=[qgc],
	gcs_system_ids={'multicolor':[1,2,3,4,5]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

15:58:58 - Oracle ⚪ - INFO - 🚀 GCS multicolor launched (PID 1904100)
15:58:58 - Oracle ⚪ - INFO - 🗺️  QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.


## Oracle checking

In [10]:
orac.run()

15:58:58 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 1 GCSs
16:00:21 - Oracle ⚪ - INFO - UAV 2 completed mission and exited
16:00:22 - Oracle ⚪ - INFO - UAV 1 completed mission and exited
16:00:23 - Oracle ⚪ - INFO - UAV 4 completed mission and exited
16:00:23 - Oracle ⚪ - INFO - UAV 3 completed mission and exited
16:00:24 - Oracle ⚪ - INFO - Received DONE from GCS multicolor
16:00:24 - Oracle ⚪ - INFO - UAV 5 completed mission and exited
16:00:24 - Oracle ⚪ - INFO - ✅ All GCS threads completed
16:00:24 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
